In [33]:
# import statements
import requests
import requests.exceptions
import base64
import json
import pandas as pd
import pprint
import sqlite3

In [35]:
# instantiate API keys
client_id = 'c06c947b53ce48a89b0dd1b7f1e7b045'
client_secret = 'f032212be7c14e7589f2d2da1d28bce4'
print('Client ID and secret instantiated!')

Client ID and secret instantiated!


In [37]:
# function for getting access token through client credentials authentication
def get_access_token(client_id, client_secret):
    # try API call
    try:
        auth_url = 'https://accounts.spotify.com/api/token'
        auth_headers = { 'Authorization': f'Basic {base64.b64encode((client_id+":"+client_secret).encode()).decode()}'}
        auth_data = {
          'grant_type': 'client_credentials',
          'client_id': client_id,
          'client_secret': client_secret,
        }

    # exception handling for API call
    except requests.exceptions.HTTPError as errh:
      return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
      return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
      return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
      return "An Unknown Error occurred: " + repr(err)

    # extract access_token from API response
    auth_response = requests.post(auth_url, headers=auth_headers, data=auth_data)
    auth_response_data = auth_response.json()
    access_token = auth_response_data['access_token']

    return access_token

In [39]:
# function to search for song
def search_song(song, artist, access_token):
    try:
        search_url = 'https://api.spotify.com/v1/search'
        search_headers = {
            'Authorization': f'Bearer {access_token}'
        }
        search_song_params = {
            'q': f'track:{song} artist:{artist}',
            'type': 'track',
            'limit': 5,
            'offset': 0
        }

        search_song_response = requests.get(search_url, headers=search_headers, params=search_song_params)

        search_artist_params = {
            'q': f'artist:{artist}',
            'type': 'artist',
            'limit': 5,
            'offset': 0
        }

        search_artist_response = requests.get(search_url, headers=search_headers, params=search_artist_params)

    except requests.exceptions.HTTPError as errh:
      return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
      return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
      return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
      return "An Unknown Error occurred: " + repr(err)


    search_song_response_data = search_song_response.json()

    search_artist_response_data = search_artist_response.json()

    if len(search_song_response_data['tracks']['items']) == 0 or len(search_artist_response_data['artists']['items']) == 0:
          return f"Song '{song}' by '{artist}' could not be found."

    genres = search_artist_response_data['artists']['items'][0]['genres']

    return search_song_response_data, genres

In [41]:
# function to get recommendations with API call
def get_recs(song_id, artist_id, genres, access_token):
    try:
        rec_url = 'https://api.spotify.com/v1/recommendations'
        rec_headers = {
            'Authorization': f'Bearer {access_token}'
        }
        rec_params = {
            'seed_tracks': song_id,
            'seed_artists': artist_id,
            'seed_genres': genres,
            'limit': 30
        }

        rec_response = requests.get(rec_url, headers=rec_headers, params=rec_params)

    except requests.exceptions.HTTPError as errh:
      return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
      return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
      return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
      return "An Unknown Error occurred: " + repr(err)

    rec_response_data = rec_response.json()

    return rec_response_data;

In [43]:
# function to get user input
def get_user_input():
    song=(input('Enter a song for recommendations: '))
    print(song)
    print()

    artist=(input('Enter artist of song: '))
    print(artist)
    print()
    return song, artist

In [45]:
# function to transform specified df columns
def transform_columns(df):
    # specify columns to be dropped
    columns_to_drop = ['album.album_type', 'album.available_markets', 'album.external_urls.spotify', 'album.href', 'album.images',
                     'album.name', 'album.release_date', 'album.release_date_precision', 'album.id', 'album.artists', 'album.total_tracks',
                     'available_markets', 'disc_number', 'duration_ms', 'explicit', 'href', 'is_local', 'id', 'track_number', 'type',
                     'uri', 'album.type', 'preview_url', 'album.uri', 'external_ids.isrc', 'popularity']

    # extract just the name from artists JSONs
    df['artists'] = df['artists'].apply(lambda x: x[0]['name'])

    # drop specified columns
    df.drop(columns=columns_to_drop, inplace=True)

    return df

In [47]:
# function for getting user input and get recommendations based on input
def main():
    # get access token
    access_token = get_access_token(client_id, client_secret)

    print("Enter a song's name and artist to receive recommendations!")
    print()

    # loop for user input prompt with invalid response handling
    while True:
        song, artist = get_user_input()

        if song == "" or artist == "":
            print("Invalid input. Please try again.")
            print()
            continue

        search_response_data = search_song(song, artist, access_token)

        if search_response_data == f"Song '{song}' by '{artist}' could not be found.":
            print(f"Song '{song}' by '{artist}' could not be found. Please try again.")
            print()
            continue

        break

    # extract song_id and artist_id for API call and gets recommendations
    search_song_response_data, genres = search_song(song, artist, access_token)
    song_id = search_song_response_data['tracks']['items'][0]['id']
    artist_id = search_song_response_data['tracks']['items'][0]['artists'][0]['id']
    song_recs_json = get_recs(song_id, artist_id, genres, access_token)

    return song_recs_json

In [49]:
# Run this to get recommendations!
song_recs_json=main()

Enter a song's name and artist to receive recommendations!



Enter a song for recommendations:  Stick Season


Stick Season



Enter artist of song:  Noah Kahan


Noah Kahan



In [51]:
# Run this to manipulate data and display most relevant columns
df = pd.json_normalize(song_recs_json['tracks'])

# Summary of file ingestion.
print(f"Number of records after file ingestion: {df.shape[0]}")
print(f"Number of columns after file ingestion: {df.shape[1]}")
print()

# Call function to transform columns
df = transform_columns(df)

# Summary of post-processing
print(f"Number of records after post-processing: {df.shape[0]}")
print(f"Number of columns after post-processing: {df.shape[1]}")
print()

Number of records after file ingestion: 30
Number of columns after file ingestion: 29

Number of records after post-processing: 30
Number of columns after post-processing: 3



In [53]:
# Display song recommendations
# Open external urls to listen
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

df

,artists,name,external_urls.spotify
0,Mon Rovîa,Big Love Ahead,https://open.spotify.com/track/3RGfmaB407YxayMDaV2jlO
1,Hozier,Like Real People Do,https://open.spotify.com/track/57V4uc2b2diZ4RPHXWecb9
2,Chris Stapleton,Starting Over,https://open.spotify.com/track/3K07bGe8iljQ3mOKArHLDo
3,Dean Lewis,Trust Me Mate,https://open.spotify.com/track/6KE2Oe4ezI1CeYCOFfdbx7
4,Chappell Roan,Red Wine Supernova,https://open.spotify.com/track/7FOgcfdz9Nx5V9lCNXdBYv
5,Brenn!,Revival,https://open.spotify.com/track/4TaRFbSPV4xhtRVPLs7BO7
6,Josiah and the Bonnevilles,Chronically Cautious,https://open.spotify.com/track/6ZrmGXr5UhSw2ymL9Bhky7
7,David Kushner,Burn,https://open.spotify.com/track/40dfo65kBJa1UgzkD652Gl
8,Caamp,Strawberries,https://open.spotify.com/track/7myzdcGa6aTaXj3Wf1V55M
9,Noah Kahan,Anyway,https://open.spotify.com/track/5FwlCxV2Q4xjTywLsD3Kxe


In [55]:
# Generate csv file
# Access file through File tab on left side of screen if using Google Colab or through Downloads if using Jupyter Notebook
df.to_csv('song_recs.csv', index=False)

In [57]:
# Generate sqlite3 db file
# Access file through File tab on left side of screen if using Google Colab or through Downloads if using Jupyter Notebook
conn = sqlite3.connect('song_recs.db')
df.to_sql('song_recs', conn, if_exists='replace', index=False)
conn.close()